In [ ]:
!pip install gpt-2-simple > log

In [ ]:
import gpt_2_simple as gpt2
import os
from datetime import datetime 
import tensorflow as tf
from google.cloud import storage

In [ ]:
model_name="124M"
checkpoint_dir = 'checkpoint'
file_name = "songs_data.txt"
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
bucket_name = "poem-generator-checkpoints"
run_name='disco_polo_gen'

In [ ]:
if not os.path.exists(os.path.join('models', model_name)):
    gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess() 
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              reuse=True,
              steps=15000,
              run_name=run_name,
              print_every=10,
              sample_every=200,
              save_every=100, 
              only_train_transformer_layers = True,
              accumulate_gradients = 1
              )

In [ ]:
bucket = storage.Client().get_bucket(bucket_name)
for f_name in os.listdir(os.path.join(checkpoint_dir, run_name)):
    bucket.blob(os.path.join(timestamp, checkpoint_dir, run_name, f_name))\
                .upload_from_filename(os.path.join(checkpoint_dir, run_name, f_name))